In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import statistics
import matplotlib.pyplot as plt
import traceback
import re
import stats
from pandas import Series
from datetime import datetime
from sklearn.model_selection import train_test_split

df = pd.read_csv('FinalWNVData v.1')

ModuleNotFoundError: No module named 'stats'

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              10506 non-null  int64  
 1   Date                    10506 non-null  object 
 2   Address                 10506 non-null  object 
 3   Species                 10506 non-null  object 
 4   Block                   10506 non-null  int64  
 5   Street                  10506 non-null  object 
 6   Trap                    10506 non-null  object 
 7   AddressNumberAndStreet  10506 non-null  object 
 8   Latitude                10506 non-null  float64
 9   Longitude               10506 non-null  float64
 10  AddressAccuracy         10506 non-null  int64  
 11  NumMosquitos            10506 non-null  int64  
 12  WnvPresent              10506 non-null  int64  
 13  Tmax                    10506 non-null  float64
 14  Tmin                    10506 non-null

# Weight of Evidence/Information Value

In [3]:
#First let's split our data into training and testing sets.
X = df.drop(columns=['WnvPresent'])
y = df['WnvPresent']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [41]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    #print(df1)
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    #print('justmiss',justmiss)
    #print('notmiss',notmiss)
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            print('d1 :',d1)
            print('d2 :',d2.mean())
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            print("I am here 1",r, n,len(d2))
            n = n - 1 
            
        except Exception as e:
            n = n - 1
            print("I am here e",n)
            sys.exit('Failed in mono_bin function')

    if len(d2) == 1:
        print("I am second step ",r, n)
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        print(justmiss.count().Y)
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    print(np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT))
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    print(df1)
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    print("hi",d3.IV )
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    x = df1.dtypes.index
    count = -1
    for i in x:
        print(i)
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                print("Number and unique value greater than 2")
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                print("I am here 2")
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [15]:
X_train.dtypes.index

Index(['Unnamed: 0', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb',
       'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'SnowFall',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'BR', 'DZ', 'FG', 'HZ', 'RA', 'TS', 'TSRA', 'VCTS', 'Time',
       'Friday', 'Monday', 'Thursday', 'Tuesday', 'Wednesday', 'August',
       'July', 'June', 'May', 'October', 'September', 'TimeSinceLastBR',
       'TimeSinceLastDZ', 'TimeSinceLastFG', 'TimeSinceLastHZ',
       'TimeSinceLastRA', 'TimeSinceLastTS', 'TimeSinceLastTSRA',
       'TimeSinceLastVCTS', 'Hyde Park Township', 'Jefferson Township',
       'Lake Township', 'Lake View Township', 'Leyden Township',
       'Norwood Park Township', 'Rogers Park Township', 'Stickney Township',
       'Thornton Township', 'Worth Township'],
     

In [17]:
X_train.dtypes.index[0].upper()

'UNNAMED: 0'

In [18]:
final.upper()

NameError: name 'final' is not defined

In [42]:
final_iv, IV = data_vars(X_train,y_train)

Unnamed: 0
Number and unique value greater than 2
d1 :          X  Y             Bucket
4115  4115  0  (3699.55, 4226.2]
9004  9004  0  (8959.05, 9483.7]
3334  3334  0  (3181.8, 3699.55]
8333  8333  0  (7891.75, 8426.4]
9021  9021  0  (8959.05, 9483.7]
...    ... ..                ...
5734  5734  0  (5296.5, 5828.15]
5191  5191  1  (4751.85, 5296.5]
5390  5390  0  (5296.5, 5828.15]
860    860  0   (530.65, 1062.3]
7270  7270  0  (6855.45, 7389.1]

[7354 rows x 3 columns]
d2 :                               X         Y
Bucket                                   
(0.999, 530.65]      265.627717  0.000000
(530.65, 1062.3]     803.605978  0.029891
(1062.3, 1609.95]   1331.272480  0.035422
(1609.95, 2135.6]   1874.472826  0.116848
(2135.6, 2653.25]   2398.092391  0.168478
(2653.25, 3181.8]   2908.166213  0.079019
(3181.8, 3699.55]   3434.035326  0.008152
(3699.55, 4226.2]   3956.103261  0.005435
(4226.2, 4751.85]   4488.907357  0.000000
(4751.85, 5296.5]   5019.717391  0.010870
(5296.5, 5828.1

NameError: name 'sys' is not defined